In [2]:
# Importing Libraries
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import GPUtil

In [3]:
# Clear Keras session
from keras import backend as K
K.clear_session()

In [4]:
import GPUtil

# Get the first GPU in the system
gpu = GPUtil.getGPUs()[0]

print(f"GPU Memory Total: {gpu.memoryTotal}MB")
print(f"GPU Memory Free: {gpu.memoryFree}MB")
print(f"GPU Memory Used: {gpu.memoryUsed}MB")
print(f"Temp: {gpu.temperature}")

GPU Memory Total: 4096.0MB
GPU Memory Free: 3721.0MB
GPU Memory Used: 228.0MB
Temp: 46.0


In [5]:
# Constants
IMG_HEIGHT, IMG_WIDTH = 124, 124  # ResNet-50 input size
BATCH_SIZE = 32

# Load data
folder = r"D:\Breast Cancer Detection - Minor Project\BreakHis dataset\BreaKHis_v1\BreaKHis_v1\histology_slides\breast"
folder_path = pathlib.Path(folder)

In [6]:
# Data preprocessing
train_generator = ImageDataGenerator(
    rescale=1./255, 
    brightness_range=[0.8, 1.2], 
    zoom_range=[.99, 1.01], 
    data_format="channels_last", 
    fill_mode="constant", 
    horizontal_flip=True
)

train_data_gen = train_generator.flow_from_directory(
    directory=folder, 
    target_size=(IMG_HEIGHT, IMG_WIDTH), 
    batch_size=6500, 
    shuffle=False
)

train_data, train_labels = train_data_gen.next()

###  ###
'''
# Synthetic Minority Over-sampling Technique (SMOTE)
sm = SMOTE(random_state=42)
train_data_flattened = train_data.reshape(train_data.shape[0], -1)
train_data_resampled, train_labels_resampled = sm.fit_resample(train_data_flattened, train_labels)
train_data_resampled = train_data_resampled.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)
train_labels_resampled = train_labels_resampled.reshape(-1, 1)
'''


# Synthetic Minority Over-sampling Technique
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_HEIGHT * IMG_WIDTH * 3), train_labels)
train_data = train_data.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)
train_labels = train_labels.reshape(-1, 1)

# Train-test-validation split
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

Found 7909 images belonging to 2 classes.


In [7]:
# Check GPU availability
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Set TensorFlow to use GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        # Enable memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is ready")
    except RuntimeError as e:
        print(e)

In [8]:
# Define and compile the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze more layers
for layer in base_model.layers[:-30]:  # Unfreeze the last 30 layers
    layer.trainable = True


# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)


In [9]:
# Train the model
history = model.fit(
    train_data, 
    train_labels, 
    epochs=10,  # Increased epochs
    batch_size=BATCH_SIZE, 
    validation_data=(val_data, val_labels),
    callbacks=[lr_scheduler, EarlyStopping(patience=5, restore_best_weights=True)]
)


Epoch 1/10
161/161 [==============================] - 848s 5s/step - loss: 0.2638 - accuracy: 0.8828 - val_loss: 5.3101 - val_accuracy: 0.4981 - lr: 1.0000e-04
Epoch 2/10
161/161 [==============================] - 755s 5s/step - loss: 0.0817 - accuracy: 0.9695 - val_loss: 1.6075 - val_accuracy: 0.5019 - lr: 1.0000e-04
Epoch 3/10
161/161 [==============================] - 647s 4s/step - loss: 0.0646 - accuracy: 0.9776 - val_loss: 1.3869 - val_accuracy: 0.4779 - lr: 1.0000e-04
Epoch 4/10
161/161 [==============================] - 810s 5s/step - loss: 0.0372 - accuracy: 0.9883 - val_loss: 3.8963 - val_accuracy: 0.5019 - lr: 1.0000e-04
Epoch 5/10
161/161 [==============================] - 620s 4s/step - loss: 0.0292 - accuracy: 0.9926 - val_loss: 5.1239 - val_accuracy: 0.5019 - lr: 1.0000e-04
Epoch 6/10
161/161 [==============================] - 616s 4s/step - loss: 0.0428 - accuracy: 0.9852 - val_loss: 3.8595 - val_accuracy: 0.5478 - lr: 1.0000e-04
Epoch 7/10
161/161 [====================

In [11]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc * 100:.2f}%')

51/51 [==============================] - 70s 1s/step - loss: 13.6780 - accuracy: 0.6567
Test accuracy: 65.67%


In [12]:
# Predict on a new image
def classify_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, 0)  # Add batch dimension
    
    prediction = model.predict(image)
    return prediction
